In [2]:
import pandas as pd
import os

from fuzzywuzzy import process

In [3]:
os.chdir("../")

In [28]:
def get_matching_name(row, df, column):
    name = row[column].lower()
    choices = df[column].str.lower().tolist()
    match, score = process.extractOne(name, choices)
    return match if score >= 80 else None  # Adjust the score threshold as needed


def check_date_overlap(row_booking, df_felho):
    start_booking = row_booking["first_night"]
    end_booking = row_booking["last_night"]
    overlap = df_felho[
        (df_felho["first_night"] <= end_booking)
        & (df_felho["last_night"] >= start_booking)
    ]
    return not overlap.empty

def highlight_rows(row):
    pastel_green = 'background-color: #d4edda; color: black;'
    pastel_yellow = 'background-color: #fff3cd; color: black;'
    pastel_red = 'background-color: #f8d7da; color: black;'
    default = [''] * len(row)
    
    if row['colour_code'] == 'green':
        return [pastel_green] * len(row)
    elif row['colour_code'] == 'yellow':
        return [pastel_yellow] * len(row)
    elif row['colour_code'] == 'red':
        return [pastel_red] * len(row)
    else:
        return default

In [45]:
df_felho = pd.read_excel("./data/felho.xlsx")
df_booking = pd.read_excel("./data/booking.xls")

In [46]:
# First dataframe columns
FELHO_COLUMNS = [
    "guestbook_id",
    "first_night",
    "last_night",
    "last_name",
    "first_name",
    "date_of_birth",
    "id_card",
    "passport",
    "driver_license",
    "other_id",
    "citizenship",
    "country_of_residence",
    "postal_code",
    "address",
    "ifa_status",
    "ifa_relevant_nights",
    "ifa_total",
    "ifa_per_night",
    "reservation",
    "arrival",
    "departure",
    "nights",
    "category",
    "residential_unit",
    "group",
    "place_of_birth",
    "email",
    "company_name",
]

# Second dataframe columns (with related columns commented)
BOOKING_COLUMNS = [
    "reservation_number",  # No related column
    "booked_by",  # No related column
    "guest_names",  # No related column
    "first_night",  # Related to 'first_night' in first list
    "last_night",  # Related to 'last_night' in first list
    "reservation_date",  # No related column
    "status",  # No related column
    "rooms",  # No related column
    "number_of_guests",  # No related column
    "adults",  # No related column
    "children",  # No related column
    "children_ages",  # No related column
    "price",  # No related column
    "commission",  # No related column
    "commission_rate",  # No related column
    "payment_status",  # No related column
    "payment_method",  # No related column
    "notes",  # No related column 
    "booking_group",  # No related column
    "booker_country",  # No related column 
    "purpose_of_travel",  # No related column 
    "device",  # No related column
    "accommodation_type",  # No related column
    "duration_nights",  # No related column
    "cancellation_date",  # No related column
    "address",  # Related to 'address' in first list
    "phone_number",  # No related column
]



In [47]:
def compare_data(df_felho,df_booking):
    df_felho.columns = FELHO_COLUMNS
    df_booking.columns = BOOKING_COLUMNS
    df_booking[["last_name", "first_name"]] = df_booking["booked_by"].str.split(
        ", ", expand=True
    )

    df_booking.insert(3, "last_name", df_booking.pop("last_name"))


    df_booking.insert(4, "first_name", df_booking.pop("first_name"))
    df_booking = df_booking[df_booking["status"] == "ok"]
    df_felho["full_name"] = df_felho["first_name"] + " " + df_felho["last_name"]
    df_booking["full_name"] = df_booking["first_name"] + " " + df_booking["last_name"]
    df_booking["first_night"] = pd.to_datetime(df_booking["first_night"])
    df_booking["last_night"] = pd.to_datetime(df_booking["last_night"])
    df_felho["first_night"] = pd.to_datetime(df_felho["first_night"])
    df_felho["last_night"] = pd.to_datetime(df_felho["last_night"])
    df_booking["full_name_match"] = df_booking.apply(
        lambda row: get_matching_name(row, df_felho, "full_name"), axis=1
    )
    df_booking["has_name_overlap"] = df_booking["full_name_match"].notna()
    df_booking["has_date_overlap"] = df_booking.apply(
        lambda row: check_date_overlap(row, df_felho), axis=1
    )
    comparing_df = df_booking.loc[
        ~df_booking["has_date_overlap"] | ~df_booking["has_name_overlap"],
        [
            "full_name",
            "first_night",
            "last_night",
            "has_name_overlap",
            "has_date_overlap",
            "number_of_guests",
            "children",
            "children_ages",
            "phone_number",
            "address",
        ],
    ]
    df_booking["colour_code"] = df_booking.apply(
        lambda row: (
            "red"
            if not row["has_date_overlap"]
            else "yellow" if not row["has_name_overlap"] else "green"
        ),
        axis=1,
    )
    comparing_df["colour_code"] = comparing_df.apply(
        lambda row: (
            "red"
            if not row["has_date_overlap"]
            else "yellow" if not row["has_name_overlap"] else "green"
        ),
        axis=1,
    )
    df_booking["colour_code"] = df_booking.apply(
        lambda row: (
            "red"
            if not row["has_date_overlap"]
            else "yellow" if not row["has_name_overlap"] else "green"
        ),
        axis=1,
    )
    df_booking = df_booking[
        [
            "full_name",
            "first_night",
            "last_night",
            "has_name_overlap",
            "has_date_overlap",
            "number_of_guests",
            "children",
            "children_ages",
            "phone_number",
            "address",
            "colour_code",
        ]
    ].style.apply(highlight_rows, axis=1).drop(columns=["colour_code"])
    return df_booking

In [48]:
compare_data(df_felho,df_booking)

C:\Users\ijara_go12\AppData\Local\Temp\ipykernel_2488\544656699.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_booking["full_name"] = df_booking["first_name"] + " " + df_booking["last_name"]
C:\Users\ijara_go12\AppData\Local\Temp\ipykernel_2488\544656699.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_booking["first_night"] = pd.to_datetime(df_booking["first_night"])
C:\Users\ijara_go12\AppData\Local\Temp\ipykernel_2488\544656699.py:16: SettingWithCopyWarning: 
A value is trying to be set o

AttributeError: 'Styler' object has no attribute 'drop'

In [28]:
df_booking[["has_date_overlap", "has_name_overlap"]].value_counts()

has_date_overlap  has_name_overlap
True              True                10
                  False                4
False             False                3
Name: count, dtype: int64